In [1]:
import pyspark
import pylint

## Using DataFrames
Spark's core data structure is the Resilient Distributed Dataset (RDD). This is a low level object that lets Spark work its magic by splitting data across multiple nodes in the cluster. However, RDDs are hard to work with directly, so in this course you'll be using the Spark DataFrame abstraction built on top of RDDs.

The Spark DataFrame was designed to behave a lot like a SQL table (a table with variables in the columns and observations in the rows). Not only are they easier to understand, DataFrames are also more optimized for complicated operations than RDDs.

When you start modifying and combining columns and rows of data, there are many ways to arrive at the same result, but some often take much longer than others. When using RDDs, it's up to the data scientist to figure out the right way to optimize the query, but the DataFrame implementation has much of this optimization built in!

To start working with Spark DataFrames, you first have to create a `SparkSession` object from your `SparkContext`. You can think of the `SparkContext` as your connection to the cluster and the `SparkSession` as your interface with that connection.

Remember, for the rest of this course you'll have a `SparkSession` called spark available in your workspace!

In [2]:
sc = pyspark.SparkContext()

In [3]:
sc.version

'3.4.0'

In [4]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Creating a SparkSession
We've already created a SparkSession for you called spark, but what if you're not sure there already is one? Creating multiple SparkSessions and SparkContexts can cause issues, so it's best practice to use the SparkSession.builder.getOrCreate() method. This returns an existing SparkSession if there's already one in the environment, or creates a new one if necessary!

* Import SparkSession from pyspark.sql.
* Make a new SparkSession called my_spark using SparkSession.builder.getOrCreate().
* Print my_spark to the console to verify it's a SparkSession.

In [5]:
from pyspark.sql import SparkSession

# Create my_spark
spark = SparkSession(sc).builder.getOrCreate()

# Print my_spark
print(spark)

In [6]:
spark

## Viewing tables
Once you've created a `SparkSession`, you can start poking around to see what data is in your cluster!

Your `SparkSession` has an attribute called catalog which lists all the data inside the cluster. This attribute has a few methods for extracting different pieces of information.

One of the most useful is the `.listTables()` method, which returns the names of all the tables in your cluster as a list.

* See what tables are in your cluster by calling `spark.catalog.listTables()` and printing the result!

### Problems running Hadoop on Windows
Hadoop requires native libraries on Windows to work properly -that includes to access the file:// filesystem, where Hadoop uses some Windows APIs to implement posix-like file access permissions.

This is implemented in HADOOP.DLL and WINUTILS.EXE.

In particular, %HADOOP_HOME%\BIN\WINUTILS.EXE must be locatable.

If it is not, Hadoop or an application built on top of Hadoop will fail.

How to fix a missing WINUTILS.EXE
You can fix this problem in two ways

Install a full native windows Hadoop version. The ASF does not currently (September 2015) release such a version; releases are available externally.
Or: get the WINUTILS.EXE binary from a Hadoop redistribution. There is a repository of this for some Hadoop versions on github.
Then

Set the environment variable %HADOOP_HOME% to point to the directory above the BIN dir containing WINUTILS.EXE.
Or: run the Java process with the system property hadoop.home.dir set to the home directory.

Refer to: [WindowsProblems](https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems) page on colfluence

#### Steps to resolve:
* Clone [this repo](https://github.com/steveloughran/winutils) into a folder using:
`git clone https://github.com/steveloughran/winutils.git`
* Add a new system variable called `HADOOP_HOME` with value of:
    `Path you cloned the repo to` +`\hadoop-3.0.0`
   Or depending on the latest version you can find
   
##### You can try this also:
* run: `pip install pyhadoop`

In [7]:
# Print the tables in the catalog
print(spark.catalog.listTables())
spark.catalog.listTables()

[]


[]

## Are you query-ious?
One of the advantages of the DataFrame interface is that you can run SQL queries on the tables in your Spark cluster. If you don't have any experience with SQL, don't worry, we'll provide you with queries! (To learn more SQL, start with our [Introduction to SQL](https://www.datacamp.com/courses/intro-to-sql-for-data-science)  course.)

As you saw in the last exercise, one of the tables in your cluster is the `flights` table. This table contains a row for every flight that left Portland International Airport (PDX) or Seattle-Tacoma International Airport (SEA) in 2014 and 2015.

Running a query on this table is as easy as using the `.sql()` method on your `SparkSession`. This method takes a string containing the query and returns a DataFrame with the results!

If you look closely, you'll notice that the table `flights` is only mentioned in the query, not as an argument to any of the methods. This is because there isn't a local object in your environment that holds that data, so it wouldn't make sense to pass the table as an argument.

Remember, we've already created a `SparkSession` called `spark` in your workspace. (It's no longer called my_spark because we created it for you!)

Use the `.sql()` method to get the first 10 rows of the `flights` table and save the result to `flights10`. The variable query contains the appropriate SQL query.
Use the DataFrame method `.show()` to print `flights10`.

In [8]:
# Don't change this query
query = "FROM flights SELECT * LIMIT 10"

# Get the first 10 rows of flights
flights10 = spark.sql(query)

# Show the results
flights10.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `flights` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 5;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Project [*]
      +- 'UnresolvedRelation [flights], [], false
